In [246]:
from django.db import models
from django.db.models.sql.query import Query
from django.db.models.query import ModelIterable


class IncludeModelIterable(ModelIterable):
    
    def __iter__(self):
        print(self.queryset._includes)
        for instance in super(IncludeModelIterable, self).__iter__():
            for includer in self.queryset._includes.values():
                includer.extract(instance)
                
            yield instance
            
            
class AbstractIncluder(object):
    def __init__(self, alias, field):
        self.alias = alias
        self.field = field
        self.model = field.related_model
        self.host_model = field.model
        

class ManyToOneIncluder(AbstractIncluder):
    
    def extract(self, instance):
        data = getattr(instance, self.alias)
        delattr(instance, self.alias)
        setattr(instance, self.field.get_cache_name(), self.model.from_db(instance._state.db, None, data))
        
    def extra_query(self):
        return {self.alias:'''
        SELECT json_build_array({select})
        FROM "{table}" AS "{alias}"
        WHERE "{alias}"."{join_column}" = "{host}"."{host_join_column}"
        '''.format(
            alias=self.alias,
            table=self.model._meta.db_table,
            join_column=self.field.target_field.column,
            host=self.host_model._meta.db_table,
            host_join_column=self.field.column,
            select=', '.join('"{}"."{}"'.format(self.alias, f.column) for f in self.model._meta.concrete_fields)
        )}


class OneToManyIncluder(AbstractIncluder):
    
    def extract(self, instance):
        data = getattr(instance, self.alias)
        delattr(instance, self.alias)
        
        if not hasattr(instance, '_prefetched_objects_cache'):
            instance._prefetched_objects_cache = {}
        instance._prefetched_objects_cache[self.field.name] = self.model.objects.none()
        instance._prefetched_objects_cache[self.field.name]._result_cache = [
            self.model.from_db(instance._state.db, None, datum)
            for datum in data
        ]
    
    def extra_query(self):
        host_join_column, join_column = self.field.get_joining_columns()[0]
        return {self.alias:'''
        SELECT json_agg(json_build_array({select}))
        FROM "{table}" AS "{alias}"
        WHERE "{alias}"."{join_column}" = "{host}"."{host_join_column}"
        '''.format(
            alias=self.alias,
            table=self.model._meta.db_table,
            join_column=join_column,
            host=self.host_model._meta.db_table,
            host_join_column=host_join_column,
            select=', '.join('"{}"."{}"'.format(self.alias, f.column) for f in self.model._meta.concrete_fields)
        )}
    
    
class OneToManyIncluder(AbstractIncluder):
    
    def extract(self, instance):
        data = getattr(instance, self.alias)
        delattr(instance, self.alias)
        
        if not hasattr(instance, '_prefetched_objects_cache'):
            instance._prefetched_objects_cache = {}
        instance._prefetched_objects_cache[self.field.name] = self.model.objects.none()
        instance._prefetched_objects_cache[self.field.name]._result_cache = [
            self.model.from_db(instance._state.db, None, datum)
            for datum in data
        ]
    
    def extra_query(self):
        host_join_column, join_column = self.field.get_joining_columns()[0]
        return {self.alias:'''
        SELECT json_agg(json_build_array({select}))
        FROM "{table}" AS "{alias}"
        WHERE "{alias}"."{join_column}" = "{host}"."{host_join_column}"
        '''.format(
            alias=self.alias,
            table=self.model._meta.db_table,
            join_column=join_column,
            host=self.host_model._meta.db_table,
            host_join_column=host_join_column,
            select=', '.join('"{}"."{}"'.format(self.alias, f.column) for f in self.model._meta.concrete_fields)
        )}
    

class IncludeQuerySet(models.QuerySet):

    def __init__(self, *args, **kwargs):
        super(BetterQuerySet, self).__init__(*args, **kwargs)
        self._includes = {}
        self._iterable_class = IncludeModelIterable

    def include(self, *related_names):
        clone = self._clone()
        for related_name in related_names:
            field = self.model._meta.get_field(related_name)
            if field.many_to_one:
                clone._includes[field] = ManyToOneIncluder('__garbage__' + field.name, field)
            elif field.one_to_many:
                clone._includes[field] = OneToManyIncluder('__garbage__' + field.name, field)
        clone.__construct_extra()
        return clone
    
    def __construct_extra(self):
        for includer in self._includes.values():
            self.query.add_extra(includer.extra_query(), None, None, None, None, None)
            
    def _clone(self):
        clone = super(BetterQuerySet, self)._clone()
        clone._includes = self._includes
        return clone
        
IncludeManager = IncludeQuerySet.as_manager()
IncludeManager.contribute_to_class(AbstractNode, 'bobjects')

In [228]:
list(AbstractNode.bobjects.all().include('creator')[:5])

{<django.db.models.fields.related.ForeignKey: creator>: <__main__.ManyToOneIncluder object at 0x111103f50>}


[<Registration: Proj : (e6248)>,
 <Registration: a registration project : (e5w73)>,
 <Registration: Comp2 : (e5pkg)>,
 <Registration: Clouds : (e5gcm)>,
 <Registration: Comp3 : (e59xc)>]

In [242]:
AbstractNode.bobjects.all().include('contributor_set')[0].contributor_set.all()

{<ManyToOneRel: osf.contributor>: <__main__.OneToManyIncluder object at 0x110f73550>}


[<Contributor(user=s.chrisinger@gmail.com, read=True, write=True, admin=True, visible=True)>]

In [230]:
print(str(AbstractNode.bobjects.all().include('contributor_set__user').query))

SELECT (
        SELECT json_agg(json_build_array("__garbage__contributor_set"."id", "__garbage__contributor_set"."read", "__garbage__contributor_set"."write", "__garbage__contributor_set"."admin", "__garbage__contributor_set"."visible", "__garbage__contributor_set"."user_id", "__garbage__contributor_set"."node_id", "__garbage__contributor_set"."_order"))
        FROM "osf_contributor" AS "__garbage__contributor_set"
        WHERE "__garbage__contributor_set"."node_id" = "osf_abstractnode"."id"
        ) AS "__garbage__contributor_set", "osf_abstractnode"."id", "osf_abstractnode"."guid_string", "osf_abstractnode"."content_type_pk", "osf_abstractnode"."type", "osf_abstractnode"."spam_status", "osf_abstractnode"."spam_pro_tip", "osf_abstractnode"."spam_data", "osf_abstractnode"."date_last_reported", "osf_abstractnode"."reports", "osf_abstractnode"."category", "osf_abstractnode"."child_node_subscriptions", "osf_abstractnode"."creator_id", "osf_abstractnode"."date_created", "osf_abstractno

In [231]:
AbstractNode.bobjects.all().include('contributor_set')[0].__dict__

{<ManyToOneRel: osf.contributor>: <__main__.OneToManyIncluder object at 0x110f73cd0>}


{'_has_abandoned_preprint': False,
 '_is_preprint_orphan': False,
 '_is_templated_clone': False,
 '_original_state': {},
 '_parent': None,
 '_prefetched_objects_cache': {u'contributor_set': [<Contributor(user=s.chrisinger@gmail.com, read=True, write=True, admin=True, visible=True)>]},
 '_state': <django.db.models.base.ModelState at 0x109f77290>,
 'category': u'project',
 'child_node_subscriptions': {},
 'content_type_pk': 27,
 'creator_id': 97,
 'date_created': datetime.datetime(2015, 7, 1, 15, 40, 23, 20000, tzinfo=<UTC>),
 'date_last_reported': None,
 'date_modified': datetime.datetime(2015, 7, 1, 15, 40, 23, 20000, tzinfo=<UTC>),
 'deleted_date': None,
 'description': u'',
 'embargo_id': None,
 'embargo_termination_approval_id': None,
 'file_guid_to_share_uuids': {},
 'forked_date': None,
 'forked_from_id': None,
 'guid_string': [u'e6248'],
 'id': 156990,
 'is_bookmark_collection': None,
 'is_deleted': False,
 'is_fork': False,
 'is_public': True,
 'keenio_read_key': u'befcd1e4b9cc7

In [232]:
AbstractNode.bobjects.all().include('guids')[0].__dict__

{<django.contrib.contenttypes.fields.GenericRelation: guids>: <__main__.OneToManyIncluder object at 0x110f73110>}


ProgrammingError: column osf_abstractnode.object_id does not exist
LINE 4:         WHERE "__garbage__guids"."id" = "osf_abstractnode"."...
                                                ^


In [122]:
x.related_model

osf.models.user.OSFUser

In [184]:
x.many_to_one

True

In [240]:
AbstractNode._meta.get_field('contributor_set').__dict__

{'field': <django.db.models.fields.related.ForeignKey: node>,
 'field_name': u'id',
 'hidden': False,
 'limit_choices_to': {},
 'many_to_one': False,
 'model': osf.models.node.AbstractNode,
 'multiple': True,
 'name': u'contributor_set',
 'on_delete': <function django.db.models.deletion.CASCADE>,
 'one_to_many': True,
 'parent_link': False,
 'related_model': osf.models.contributor.Contributor,
 'related_name': u'contributor_set',
 'related_query_name': None,
 'symmetrical': False}

In [244]:
field = AbstractNode._meta.get_field('_contributors')

In [245]:
field.__dict__

{'_error_messages': None,
 '_unique': False,
 '_validators': [],
 '_verbose_name': None,
 'attname': '_contributors',
 'auto_created': False,
 'blank': False,
 'choices': [],
 'column': '_contributors',
 'concrete': True,
 'creation_counter': 201,
 'db_column': None,
 'db_index': False,
 'db_table': None,
 'db_tablespace': '',
 'default': <class django.db.models.fields.NOT_PROVIDED at 0x10c5ac600>,
 'editable': True,
 'error_messages': {u'blank': <django.utils.functional.__proxy__ at 0x10c5b5ed0>,
  u'invalid_choice': <django.utils.functional.__proxy__ at 0x10c5a8b50>,
  u'null': <django.utils.functional.__proxy__ at 0x10c5b5e90>,
  u'unique': <django.utils.functional.__proxy__ at 0x10c5b5f10>,
  u'unique_for_date': <django.utils.functional.__proxy__ at 0x10c5b5f50>},
 'has_null_arg': False,
 'help_text': u'',
 'is_relation': True,
 'm2m_column_name': <function django.utils.functional._curried>,
 'm2m_db_table': <function django.utils.functional._curried>,
 'm2m_field_name': <function 

In [239]:
field.get_joining_columns()

(('object_id', u'id'),)

In [257]:
q = Query(Contributor)
q.set_select('id')
q.sql_with_params()

AttributeError: 'str' object has no attribute 'as_sql'

(u'SELECT "osf_contributor"."id", "osf_contributor"."read", "osf_contributor"."write", "osf_contributor"."admin", "osf_contributor"."visible", "osf_contributor"."user_id", "osf_contributor"."node_id", "osf_contributor"."_order" FROM "osf_contributor" ORDER BY "osf_contributor"."_order" ASC',
 ())

In [254]:
q.__dict__

{'_annotation_select_cache': None,
 '_annotations': OrderedDict(),
 '_extra': None,
 '_extra_select_cache': None,
 'alias_map': {u'osf_contributor': <django.db.models.sql.datastructures.BaseTable at 0x109f77a90>},
 'alias_refcount': {u'osf_contributor': 0},
 'annotation_select_mask': None,
 'context': {},
 'default_cols': True,
 'default_ordering': True,
 'deferred_loading': (set(), True),
 'distinct': False,
 'distinct_fields': [],
 'external_aliases': set(),
 'extra_order_by': (),
 'extra_select_mask': None,
 'extra_tables': (),
 'filter_is_sticky': False,
 'group_by': None,
 'high_mark': None,
 'low_mark': 0,
 'max_depth': 5,
 'model': osf.models.contributor.Contributor,
 'order_by': [],
 'select': [],
 'select_for_update': False,
 'select_for_update_nowait': False,
 'select_related': False,
 'standard_ordering': True,
 'table_map': {u'osf_contributor': [u'osf_contributor']},
 'tables': [u'osf_contributor'],
 'used_aliases': set(),
 'values_select': [],
 'where': <WhereNode: (AND: )